# Object Following - Live Demo

### Control robot to follow central object

In [1]:
from jetbot import ObjectDetector
import tensorrt
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

print('ok')

ok


### Let's display all the control widgets

In [17]:
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg

from IPython.display import display
import ipywidgets.widgets as widgets

import time

robot = Robot()
camera = Camera.instance(width=300, height=300, capture_width=300, capture_height=300, fps=10)
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=1, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=47, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.25, min=0.0, max=0.5, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.25, min=0.0, max=0.5, description='turn gain')
width = int(image_widget.width)
height = int(image_widget.height)

wd_label = widgets.FloatText(value=0, description='left')
ht_label = widgets.FloatText(value=0, description='right')
occur_label = widgets.FloatText(value=0, description='occurence')

button_layout = widgets.Layout(width='128px', height='64px')
stop_button = widgets.Button(description='STOP', button_style='danger', layout=button_layout)
stop_button.on_click(lambda x: stop())

display(widgets.HBox([image_widget, widgets.VBox([blocked_widget, speed_widget, turn_gain_widget,stop_button])]))
display(widgets.VBox([label_widget,wd_label,ht_label,occur_label]))

run()

ok


### Connect the network execution function to the camera updates

In [15]:
from collections import deque
import numpy as np
occur_dq = deque([0]*10)

fwdtq = 0.4
turntq = 0.35

def stop():
    camera.unobserve_all()
    time.sleep(1.0)
    robot.stop()
    print('stopped')

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

tt = time.time()
def execute(change):
    global tt
    if time.time() - tt < 0.1: return #max update per x seconds
    else: tt = time.time()

    occur_label.value = sum(occur_dq) #how many times in the last xx sec did we see it?
    image = change['new']

    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked

    # stop if blocked
    if prob_blocked > 0.5:
        label_widget.description = "blocked"
        robot.stop()
        image_widget.value = bgr8_to_jpeg(image)
        return

    # compute all detected objects
    detections = model(image)

    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)

    #go forward if no target detected
    if det is None:
        label_widget.description = "not found"
        occur_dq.append(0)
        occur_dq.popleft()

        if sum(occur_dq) > 0: #just saw it, maybe it's still there
            # update image widget
            image_widget.value = bgr8_to_jpeg(image)
            return

        wd_label.value = 0
        ht_label.value = 0
        robot.forward(fwdtq)
        time.sleep(0.001)
        robot.left_motor.value = speed_widget.value * 1.05
        robot.right_motor.value = speed_widget.value
        # update image widget
        image_widget.value = bgr8_to_jpeg(image)
        return

    #target is detected
    else: 
        bbox = det['bbox']
        wd_label.value = round(bbox[0],2) #round(abs(bbox[0] - bbox[2]),2)
        ht_label.value = round(bbox[2],2) #round(abs(bbox[1] - bbox[3]),2)
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)

        #center about target if not centered
        if bbox[0] > 0.5 or bbox[2] < 0.5:
            label_widget.description = "centering"
            # move robot forward and steer proportional target's x-distance from center
            center = detection_center(det)              
            sign = np.sign(center[0])
            robot.left_motor.value = turntq * sign
            robot.right_motor.value = -turntq * sign
            time.sleep(0.001)
            robot.left_motor.value = turn_gain_widget.value * sign
            robot.right_motor.value = -turn_gain_widget.value * sign

            # update image widget
            image_widget.value = bgr8_to_jpeg(image)
            return
        #move forward if too small
        elif bbox[2] - bbox[0] < 0.1:
            label_widget.description = "closing in"
            robot.forward(fwdtq)
            time.sleep(0.001)
            robot.left_motor.value = speed_widget.value * 1.05
            robot.right_motor.value = speed_widget.value
        #stop if closed and centered
        else:
            label_widget.description = "found"
            occur_dq.append(1)
            occur_dq.popleft()
            robot.stop()
            # update image widget
            image_widget.value = bgr8_to_jpeg(image)
            return

def run():
    execute({'new': camera.value})
    camera.unobserve_all()
    camera.observe(execute, names='value')

### Stop

In [10]:
stop()

stopped
